In [1]:
import os
import warnings
from tqdm import tqdm
import copy
from PIL import Image
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision import transforms
import random

from networks.dan import DAN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\Darkl\anaconda3\envs\mtcnn\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\Darkl\anaconda3\envs\mtcnn\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
batch_size=32
num_head=4  # Number of attention head.
training_size = 28317 + 3541

image_path = '../dataset/images.npy'
training_emotion_path = "../dataset/emotions_1.npy"
test_emotion_path = '../dataset/emotions_multi.npy'

In [3]:
def warn(*args, **kwargs):
    pass
warnings.warn = warn

'''
def load_data(image_path, training_path, test_path, subset):
    images = np.load(image_path)        # shape = (35393, 48, 48, 1)
    # images = images/255.0
    images = np.float32(images)
    
    if subset == 'train':
        training_emotions = np.load(training_path)    # shape = (35393, 8)
        training_emotions = np.float32(training_emotions)
        return images[:training_size], training_emotions[:training_size]
    if subset == 'test':
        test_emotions = np.load(test_path)
        test_emotions = np.float32(test_emotions)
        return images[training_size:], test_emotions[training_size:]
'''

sample_size = 35393
training_size = 28317 + 3541
emotions = np.load(test_emotion_path)
test_subset_indices = []
for i in range(sample_size):
    if np.count_nonzero(emotions[i] > 0) >= 4:
        test_subset_indices.append(i)
test_subset_indices = random.sample(test_subset_indices, k=sample_size-training_size)
train_subset_indices = [item for item in range(sample_size) if not item in test_subset_indices]

def load_data(image_path, training_path, test_path, subset):
    images = np.load(image_path)   
    images = np.float32(images)
    if subset == 'train':
        training_emotions = np.load(training_path) 
        training_emotions = np.float32(training_emotions)
        return images[train_subset_indices], training_emotions[train_subset_indices]
    if subset == 'test':
        test_emotions = np.load(test_path)
        test_emotions = np.float32(test_emotions)
        return images[test_subset_indices], test_emotions[test_subset_indices]

In [4]:
class FERPlusDataset(data.Dataset):
    def __init__(self, image_path, training_path, test_path, subset, transform = None):
        self.transform = transform
        assert(subset=='train' or subset=='test')
        self.images, self.emotions = load_data(image_path, training_path, test_path, subset)

    def __getitem__(self, index):
        image = self.images[index]
        emotion = self.emotions[index]
        if self.transform is not None:
            image = self.transform(image)
        return image, emotion
    
    def __len__(self):
        return len(self.images)

In [5]:
class AffinityLoss(nn.Module):
    def __init__(self, device, num_class=8, feat_dim=512):
        super(AffinityLoss, self).__init__()
        self.num_class = num_class
        self.feat_dim = feat_dim
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.device = device

        self.centers = nn.Parameter(torch.randn(self.num_class, self.feat_dim).to(device))

    def forward(self, x, labels):
        x = self.gap(x).view(x.size(0), -1)

        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_class) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_class, batch_size).t()
        distmat.addmm_(x, self.centers.t(), beta=1, alpha=-2)

        classes = torch.arange(self.num_class).long().to(self.device)
        labels = labels.expand(batch_size, self.num_class)
        mask = labels.eq(classes.expand(batch_size, self.num_class))

        dist = distmat * mask.float()
        dist = dist / self.centers.var(dim=0).sum()

        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size

        return loss

In [6]:
class PartitionLoss(nn.Module):
    def __init__(self, ):
        super(PartitionLoss, self).__init__()
    
    def forward(self, x):
        num_head = x.size(1)

        if num_head > 1:
            var = x.var(dim=1).mean()
            loss = torch.log(1+num_head/var)
        else:
            loss = 0
            
        return loss

In [7]:
def accuracy(output, target):
    batch_size = target.size(0)
    acc = 0
    for i in range(batch_size):
        true = target[i]
        pred = output[i]
        index_max = torch.argmax(pred)
        if true[index_max] == torch.max(true):
            acc += 1
    acc = float(acc)/batch_size
    return acc

In [8]:
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True
print("GPU",torch.cuda.is_available())

model = DAN(num_head=num_head)
model = model.to(device)

data_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomApply([
            transforms.RandomRotation(20),
            transforms.RandomCrop(224, padding=32)
        ], p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
                                
    transforms.RandomErasing(scale=(0.02,0.25)),
    ])    
train_dataset = FERPlusDataset(image_path=image_path, 
                               training_path=training_emotion_path,
                               test_path=test_emotion_path,
                               subset="train", 
                               transform=data_transforms)    
print('Whole train set size:', train_dataset.__len__())
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

data_transforms_val = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])
                                   
val_dataset = FERPlusDataset(image_path=image_path, 
                             training_path=training_emotion_path,
                             test_path=test_emotion_path,
                             subset="test", 
                             transform=data_transforms_val) 
print('Validation set size:', val_dataset.__len__())
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
)

loss1 = AffinityLoss(device)
loss2 = PartitionLoss()
params = list(model.parameters()) + list(loss1.parameters())

GPU True
Whole train set size: 31858
Validation set size: 3535


In [9]:
def fit(model, epochs, lr, factor1=1, factor2=1, adam=False, mse=False):
    
    best_model = None
    best_optim = None
    best_acc = 0 
    
    with torch.no_grad():        
        iter_cnt = 0
        acc = 0
        model.eval()
        for (imgs, targets) in val_loader:
            imgs = imgs.to(device)
            targets = targets.to(device)            
            out,feat,heads = model(imgs)
            iter_cnt+=1            
            correctness = accuracy(out, targets)
            acc += correctness / len(val_loader)            
        best_acc = acc
        best_model=copy.deepcopy(model.state_dict())
        print("copy best successfully!") 
        tqdm.write("Current best accuracy:%.4f." % (acc))
        
    if mse == True:
        loss0 = torch.nn.MSELoss().to(device) 
    else:
        loss0 = torch.nn.CrossEntropyLoss().to(device)

    if adam == True:
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
    else:
        optimizer = torch.optim.SGD(params,lr=lr, weight_decay = 1e-4, momentum=0.9)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    
    
    for epoch in range(1, epochs + 1):
        running_loss = 0.0
        acc = 0
        iter_cnt = 0
        model.train()

        for (imgs, targets) in tqdm(train_loader):
            iter_cnt += 1
            optimizer.zero_grad()
            
            imgs = imgs.to(device)
            targets = targets.to(device)
            out,feat,heads = model(imgs)
            
            loss = loss0(out,targets) + factor1*loss1(feat,targets) + factor2*loss2(heads)  

            loss.backward()
            optimizer.step()

            running_loss += loss
            correctness = (out.argmax(dim=1) == targets.argmax(dim=1)).float().mean()
            acc += correctness / len(train_loader)

        running_loss = running_loss/iter_cnt
        tqdm.write('[Epoch %d] Training accuracy: %.4f. Loss: %.4f. LR %.6f' % (epoch, acc, running_loss, optimizer.param_groups[0]['lr']))

        with torch.no_grad():
            running_loss = 0.0
            iter_cnt = 0
            acc = 0
            model.eval()
            
            for (imgs, targets) in tqdm(val_loader):
                imgs = imgs.to(device)
                targets = targets.to(device)
                out,feat,heads = model(imgs)
                
                loss = loss0(out,targets) + loss1(feat,targets) + loss2(heads)
                running_loss += loss
                iter_cnt+=1
                correctness = accuracy(out, targets)
                acc += correctness / len(val_loader)

            running_loss = running_loss/iter_cnt   
            #scheduler.step()

            tqdm.write("[Epoch %d] Validation accuracy:%.4f. Loss:%.4f" % (epoch, acc,  running_loss))
                            
        if best_acc < acc:
            best_acc = acc
            best_model=copy.deepcopy(model.state_dict())            
            print("copy best successfully!")    
        tqdm.write("best_acc:" + str(best_acc))
            
    if best_model is not None:
        model.load_state_dict(best_model)
        print("load best successfully!")    

In [10]:
checkpoint = torch.load('./models/affecnet8_epoch5_acc0.6209.pth')
model.load_state_dict(checkpoint['model_state_dict'])
fit(model, epochs=30, lr=1e-3, factor1=.5, factor2=.5, adam=1, mse=1)
fit(model, epochs=30, lr=1e-4, factor1=.5, factor2=.5, adam=1, mse=1)

copy best successfully!
Current best accuracy:0.1634.


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:30<00:00, 11.03it/s]


[Epoch 1] Training accuracy: 0.6635. Loss: 0.7924. LR 0.001000


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:07<00:00, 15.74it/s]


[Epoch 1] Validation accuracy:0.5279. Loss:0.4717
copy best successfully!
best_acc:0.5278528528528528


...

[Epoch 29] Validation accuracy:0.6574. Loss:0.1336
best_acc:0.6650337837837839


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:25<00:00, 11.62it/s]


[Epoch 30] Training accuracy: 0.9569. Loss: 0.3128. LR 0.000100


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:07<00:00, 15.53it/s]

[Epoch 30] Validation accuracy:0.6586. Loss:0.1279
best_acc:0.6650337837837839
load best successfully!


In [11]:
model.load_state_dict(checkpoint['model_state_dict'])
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_2.npy"
train_dataset = FERPlusDataset(image_path=image_path, 
                               training_path=training_emotion_path,
                               test_path=test_emotion_path,
                               subset="train", 
                               transform=data_transforms)    
print('Whole train set size:', train_dataset.__len__())
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
fit(model, epochs=30, lr=1e-3, factor1=.5, factor2=.5, adam=1, mse=1)
fit(model, epochs=30, lr=1e-4, factor1=.5, factor2=.5, adam=1, mse=1)

Whole train set size: 31858
copy best successfully!
Current best accuracy:0.1634.


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:25<00:00, 11.66it/s]


[Epoch 1] Training accuracy: 0.6659. Loss: 0.6612. LR 0.001000


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:06<00:00, 15.93it/s]


[Epoch 1] Validation accuracy:0.4966. Loss:0.2833
copy best successfully!
best_acc:0.4966403903903904


...

[Epoch 29] Validation accuracy:0.6819. Loss:0.1065
best_acc:0.6886448948948953


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:25<00:00, 11.60it/s]


[Epoch 30] Training accuracy: 0.9289. Loss: 0.2028. LR 0.000100


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:07<00:00, 15.57it/s]

[Epoch 30] Validation accuracy:0.6774. Loss:0.1066
best_acc:0.6886448948948953
load best successfully!


In [12]:
model.load_state_dict(checkpoint['model_state_dict'])
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_3.npy"
train_dataset = FERPlusDataset(image_path=image_path, 
                               training_path=training_emotion_path,
                               test_path=test_emotion_path,
                               subset="train", 
                               transform=data_transforms)    
print('Whole train set size:', train_dataset.__len__())
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
fit(model, epochs=30, lr=1e-3, factor1=.5, factor2=.5, adam=1, mse=1)
fit(model, epochs=30, lr=1e-4, factor1=.5, factor2=.5, adam=1, mse=1)

Whole train set size: 31858
copy best successfully!
Current best accuracy:0.1634.


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:25<00:00, 11.61it/s]


[Epoch 1] Training accuracy: 0.6608. Loss: 0.6490. LR 0.001000


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:06<00:00, 15.90it/s]


[Epoch 1] Validation accuracy:0.5321. Loss:0.2721
copy best successfully!
best_acc:0.5321133633633632


...

[Epoch 29] Validation accuracy:0.6813. Loss:0.1007
best_acc:0.6923048048048053


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:24<00:00, 11.75it/s]


[Epoch 30] Training accuracy: 0.9214. Loss: 0.1926. LR 0.000100


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:07<00:00, 15.77it/s]

[Epoch 30] Validation accuracy:0.6833. Loss:0.1010
best_acc:0.6923048048048053
load best successfully!


In [13]:
model.load_state_dict(checkpoint['model_state_dict'])
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_4.npy"
train_dataset = FERPlusDataset(image_path=image_path, 
                               training_path=training_emotion_path,
                               test_path=test_emotion_path,
                               subset="train", 
                               transform=data_transforms)    
print('Whole train set size:', train_dataset.__len__())
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
fit(model, epochs=30, lr=1e-3, factor1=.5, factor2=.5, adam=1, mse=1)
fit(model, epochs=30, lr=1e-4, factor1=.5, factor2=.5, adam=1, mse=1)

Whole train set size: 31858
copy best successfully!
Current best accuracy:0.1634.


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:25<00:00, 11.63it/s]


[Epoch 1] Training accuracy: 0.6542. Loss: 0.6482. LR 0.001000


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:06<00:00, 15.95it/s]


[Epoch 1] Validation accuracy:0.4749. Loss:0.4228
copy best successfully!
best_acc:0.47492492492492466


...

[Epoch 29] Validation accuracy:0.6878. Loss:0.1006
best_acc:0.6897710210210213


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:25<00:00, 11.59it/s]


[Epoch 30] Training accuracy: 0.9240. Loss: 0.1912. LR 0.000100


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:07<00:00, 15.76it/s]

[Epoch 30] Validation accuracy:0.6876. Loss:0.1006
best_acc:0.6897710210210213
load best successfully!


In [14]:
model.load_state_dict(checkpoint['model_state_dict'])
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_5.npy"
train_dataset = FERPlusDataset(image_path=image_path, 
                               training_path=training_emotion_path,
                               test_path=test_emotion_path,
                               subset="train", 
                               transform=data_transforms)    
print('Whole train set size:', train_dataset.__len__())
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
fit(model, epochs=30, lr=1e-3, factor1=.5, factor2=.5, adam=1, mse=1)
fit(model, epochs=30, lr=1e-4, factor1=.5, factor2=.5, adam=1, mse=1)

Whole train set size: 31858
copy best successfully!
Current best accuracy:0.1634.


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:25<00:00, 11.61it/s]


[Epoch 1] Training accuracy: 0.6619. Loss: 0.6478. LR 0.001000


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:06<00:00, 16.01it/s]


[Epoch 1] Validation accuracy:0.5262. Loss:0.2962
copy best successfully!
best_acc:0.5262012012012011


...

[Epoch 29] Validation accuracy:0.6777. Loss:0.1004
best_acc:0.6872747747747748


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:29<00:00, 11.15it/s]


[Epoch 30] Training accuracy: 0.9224. Loss: 0.1913. LR 0.000100


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:07<00:00, 14.84it/s]

[Epoch 30] Validation accuracy:0.6783. Loss:0.1005
best_acc:0.6872747747747748
load best successfully!


In [15]:
model.load_state_dict(checkpoint['model_state_dict'])
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_6.npy"
train_dataset = FERPlusDataset(image_path=image_path, 
                               training_path=training_emotion_path,
                               test_path=test_emotion_path,
                               subset="train", 
                               transform=data_transforms)    
print('Whole train set size:', train_dataset.__len__())
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
fit(model, epochs=30, lr=1e-3, factor1=.5, factor2=.5, adam=1, mse=1)
fit(model, epochs=30, lr=1e-4, factor1=.5, factor2=.5, adam=1, mse=1)

Whole train set size: 31858
copy best successfully!
Current best accuracy:0.1634.


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:29<00:00, 11.18it/s]


[Epoch 1] Training accuracy: 0.6625. Loss: 0.6461. LR 0.001000


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:07<00:00, 15.44it/s]


[Epoch 1] Validation accuracy:0.5346. Loss:0.2679
copy best successfully!
best_acc:0.5346096096096095


...

[Epoch 29] Validation accuracy:0.6847. Loss:0.1012
best_acc:0.6934684684684688


100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [01:29<00:00, 11.12it/s]


[Epoch 30] Training accuracy: 0.9223. Loss: 0.1915. LR 0.000100


100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [00:07<00:00, 14.86it/s]

[Epoch 30] Validation accuracy:0.6853. Loss:0.1006
best_acc:0.6934684684684688
load best successfully!


In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_7.npy"
train_dataset = FERPlusDataset(image_path=image_path, 
                               training_path=training_emotion_path,
                               test_path=test_emotion_path,
                               subset="train", 
                               transform=data_transforms)    
print('Whole train set size:', train_dataset.__len__())
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
fit(model, epochs=30, lr=1e-3, factor1=.5, factor2=.5, adam=1, mse=1)
fit(model, epochs=30, lr=1e-4, factor1=.5, factor2=.5, adam=1, mse=1)

Whole train set size: 31858
copy best successfully!
Current best accuracy:0.1634.


 70%|███████████████████████████████████████████████████████▋                        | 693/996 [01:03<00:27, 10.94it/s]

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
torch.cuda.empty_cache()
training_emotion_path = "../dataset/emotions_multi.npy"
train_dataset = FERPlusDataset(image_path=image_path, 
                               training_path=training_emotion_path,
                               test_path=test_emotion_path,
                               subset="train", 
                               transform=data_transforms)    
print('Whole train set size:', train_dataset.__len__())
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)
fit(model, epochs=30, lr=1e-3, factor1=.5, factor2=.5, adam=1, mse=1)
fit(model, epochs=30, lr=1e-4, factor1=.5, factor2=.5, adam=1, mse=1)

In [ ]:
'''
#model = DAN(num_head=num_head)
#model = model.to(device)
checkpoint = torch.load('./models/affecnet8_epoch5_acc0.6209.pth')
model.load_state_dict(checkpoint['model_state_dict'])

fit(model, epochs=10, lr=1e-1, factor1=1, factor2=1, adam=0, mse=0)
fit(model, epochs=10, lr=2e-2, factor1=1, factor2=1, adam=0, mse=0)
fit(model, epochs=10, lr=5e-3, factor1=1, factor2=1, adam=0, mse=0)
fit(model, epochs=10, lr=1e-3, factor1=1, factor2=1, adam=0, mse=0)
fit(model, epochs=10, lr=2e-4, factor1=1, factor2=1, adam=0, mse=0)

torch.save(model.state_dict(), './models/7-label-sgd.pth')
'''